In [41]:
import owlready2 as owl
import pandas as pd

filename_onto_individuals = "../../src/ontology/final-archi-individuals.owl"
onto_individuals = owl.get_ontology(filename_onto_individuals).load()

## Airport synonyms

In [65]:
df_airports_full = pd.read_csv('../../data/airports.csv')
df_airports_full

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
0,6523,00A,heliport,Total Rf Heliport,40.070801,-74.933601,11.0,NaN,US,US-PA,Bensalem,no,00A,NaN,00A,NaN,NaN,NaN
1,323361,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,3435.0,NaN,US,US-KS,Leoti,no,00AA,NaN,00AA,NaN,NaN,NaN
2,6524,00AK,small_airport,Lowell Field,59.947733,-151.692524,450.0,NaN,US,US-AK,Anchor Point,no,00AK,NaN,00AK,NaN,NaN,NaN
3,6525,00AL,small_airport,Epps Airpark,34.864799,-86.770302,820.0,NaN,US,US-AL,Harvest,no,00AL,NaN,00AL,NaN,NaN,NaN
4,6526,00AR,closed,Newport Hospital & Clinic Heliport,35.608700,-91.254898,237.0,NaN,US,US-AR,Newport,no,NaN,NaN,NaN,NaN,NaN,00AR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68469,46378,ZZ-0001,heliport,Sealand Helipad,51.894444,1.482500,40.0,EU,GB,GB-ENG,Sealand,no,NaN,NaN,NaN,http://www.sealandgov.org/,https://en.wikipedia.org/wiki/Principality_of_...,Roughs Tower Helipad
68470,307326,ZZ-0002,small_airport,Glorioso Islands Airstrip,-11.584278,47.296389,11.0,AF,TF,TF-U-A,Grande Glorieuse,no,NaN,NaN,NaN,NaN,NaN,NaN
68471,346788,ZZ-0003,small_airport,Fainting Goat Airport,32.110587,-97.356312,690.0,NaN,US,US-TX,Blum,no,87TX,NaN,87TX,NaN,NaN,NaN
68472,342102,ZZZW,closed,Scandium City Heliport,69.355287,-138.939310,4.0,NaN,CA,CA-YT,(Old) Scandium City,no,ZZZW,ZYW,YK96,NaN,NaN,NaN


In [42]:
all_airports = list(owl.default_world.sparql(
    f"""
        PREFIX pie:<http://www.semanticweb.org/clement/ontologies/2020/1/final-archi#>
        SELECT ?name ?iata ?icao ?latitude ?longitude ?altitude ?country
        WHERE {{
            ?Airport pie:AirportName ?name .
            ?Airport pie:AirportIATA ?iata .
            ?Airport pie:AirportICAOCode ?icao .
            ?Airport pie:AirportGPSLatitude ?latitude .
            ?Airport pie:AirportGPSLongitude ?longitude .
            ?Airport pie:AirportAltitude ?altitude .
            ?Airport pie:AirportCountry ?country .
        }}
    """))
fields = ['name', 'iata', 'icao', 'latitude', 'longitude', 'altitude', 'country']
dict_all_airports = [dict(zip(fields, airport_tuple)) for airport_tuple in all_airports]
df_airports = pd.DataFrame(dict_all_airports)

In [55]:
df_airports_crop = df_airports.loc[df_airports['country'] == 'France']\
                                .reset_index(drop=True)

In [56]:
df_airports_crop

,name,iata,icao,latitude,longitude,altitude,country
0,Agen La Garenne Airport,AGF,LFBA,44.174709,0.590619,204,France
1,Aix-en-Provence Airport,QXB,LFMA,43.505280,5.367222,368,France
2,Ajaccio Napoleon Bonaparte Airport,AJA,LFKJ,41.923882,8.802500,16,France
3,Albert Picardie Airport,BYF,LFAQ,49.971500,2.697660,364,France
4,Albi Le Sequestre Airport,LBI,LFCI,43.913269,2.116750,564,France
...,...,...,...,...,...,...,...
97,Toussus-Le-Noble Airport,TNF,LFPN,48.749748,2.111178,538,France
98,Troyes Barberey Airport,QYT,LFQB,48.321609,4.016622,388,France
99,Valence Chabeuil Airport,VAF,LFLU,44.915550,4.968744,525,France
100,Valenciennes Denain Airport,QYV,LFAV,50.326458,3.458615,170,France


In [57]:
def split_airport(name):
    return name.replace(' Airport', '')

In [58]:
short_names = df_airports_crop['name'].apply(split_airport).tolist()
full_names = df_airports_crop['name']
icao = df_airports_crop['icao']
iata = df_airports_crop['iata']

In [61]:
str_output = ""
for x in list(zip(icao, short_names, full_names, iata)):
    str_output += f"  - {list(x)}\n"

str_output = str_output.replace("'","").replace('"','')

In [62]:
print(str_output)

  - [LFBA, Agen La Garenne, Agen La Garenne Airport, AGF]
  - [LFMA, Aix-en-Provence, Aix-en-Provence Airport, QXB]
  - [LFKJ, Ajaccio Napoleon Bonaparte, Ajaccio Napoleon Bonaparte Airport, AJA]
  - [LFAQ, Albert Picardie, Albert Picardie Airport, BYF]
  - [LFCI, Albi Le Sequestre, Albi Le Sequestre Airport, LBI]
  - [LFJR, Angers Loire, Angers Loire Airport, ANE]
  - [LFBU, Angouleme Cognac International, Angouleme Cognac International Airport, ANG]
  - [LFLP, Annecy Mont Blanc, Annecy Mont Blanc Airport, NCY]
  - [LFLI, Annemasse, Annemasse Airport, QYA]
  - [LFCH, Arcachon La Teste-de-Buch, Arcachon La Teste-de-Buch Airport, XAC]
  - [LFLW, Aurillac Tronquieres, Aurillac Tronquieres Airport, AUR]
  - [LFLA, Auxerre Branches, Auxerre Branches Airport, AUF]
  - [LFMV, Avignon Caumont, Avignon Caumont Airport, AVN]
  - [LFSB, Basel Mulhouse-Freiburg EuroAirport, Basel Mulhouse-Freiburg EuroAirport, BSL]
  - [LFKB, Bastia Poretta, Bastia Poretta Airport, BIA]
  - [LFBE, Bergerac Dordog

In [64]:
output_file = "./airports_france_synonym.txt"
with open(output_file, "w", encoding="utf-8") as f:
    f.write(str_output)

## Tests Snips

In [ ]:
# Dépendances OK
# snips_nlu==0.20.2
# numpy==1.22.2
# scikit-learn==0.22.0

In [1]:
from snips_nlu import SnipsNLUEngine
from snips_nlu.default_configs import CONFIG_EN
import io
import json

In [5]:
!snips-nlu generate-dataset en requests_datasets.yaml > requests_datasets.json

In [ ]:
!snips-nlu train requests_datasets.json nlu_engine

### Import

In [11]:
# From persist folder
training_path = 'nlu_engine'
nlu_engine = SnipsNLUEngine.from_path(training_path)

In [2]:
# From bytes
trained_model = 'engine.snips'
with open(trained_model,'rb') as f:
    engine_bytes = f.read()
nlu_engine = SnipsNLUEngine.from_byte_array(engine_bytes)

In [3]:
nlu_engine.fitted

True

### Training

In [2]:
with io.open("requests_datasets.json") as f:
    sample_dataset = json.load(f)

nlu_engine = SnipsNLUEngine(config=CONFIG_EN)
nlu_engine = nlu_engine.fit(sample_dataset)

### Export

In [ ]:
# With persist folder
training_path = 'nlu_engine'
nlu_engine.persist(training_path)

In [13]:
# With bytes
engine_bytes = nlu_engine.to_byte_array()

trained_model = 'engine.snips'
with open(trained_model,'wb') as f:
    f.write(engine_bytes)

### Parsing

In [4]:
question = "what is the weather at TLS"
parsing = nlu_engine.parse(question)

In [5]:
parsing

{'input': 'what is the weather at TLS',
 'intent': {'intentName': 'weatherAirport', 'probability': 1.0},
 'slots': [{'range': {'start': 12, 'end': 19},
   'rawValue': 'weather',
   'value': {'kind': 'Custom', 'value': 'weather'},
   'entity': 'weatherInfo',
   'slotName': 'info'},
  {'range': {'start': 23, 'end': 26},
   'rawValue': 'TLS',
   'value': {'kind': 'Custom', 'value': 'LFBO'},
   'entity': 'airport',
   'slotName': 'place'}]}